I can not run on Theia so I download materials and run on local computer

In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
spacex_df['Launch Site'].value_counts()

CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: Launch Site, dtype: int64

In [4]:
# Create a dash application
app = dash.Dash(__name__)

In [5]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
#                                 html.Br(),
                                dcc.Dropdown(id='site-dropdown',
                                             options =[
                                                     {'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                     ],
                                             value = 'ALL',
                                             placeholder="select a launch site here",
                                             searchable=True
                                            ),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
#                                 html.Div(dcc.Graph(id='success-pie-chart')),
#                                 html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       100: '100',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500',
                                                       10000: '10000'},
                                                value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
#                                 html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Div(dcc.Graph(id='success-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    piechart_df = spacex_df.groupby(['Launch Site','class']).size().reset_index(name='counts')
    if entered_site == 'ALL':
        fig = px.pie(piechart_df, values='counts', 
        names='Launch Site', 
        title='all sites')
        return fig
    else:
        piechart_df2 = piechart_df.loc[piechart_df['Launch Site'] == entered_site]
        fig = px.pie(piechart_df2, values='counts', 
        names='class',             
        title='Success rate of ' + entered_site)
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site,slider_range):
    if entered_site == 'ALL':        
        low, high = slider_range
        mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
        fig = px.scatter(spacex_df[mask], x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return fig
    else:
        scatter_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        low, high = slider_range
        mask = (scatter_df['Payload Mass (kg)'] > low) & (scatter_df['Payload Mass (kg)'] < high)
        fig = px.scatter(scatter_df[mask], x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Oct/2021 20:50:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:50:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:50:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:50:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:50:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:50:47] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:50:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:50:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:50:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:51:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:51:42] "POST /_dash-

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\DELL\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\DELL\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\DELL\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\DELL\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\DELL\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\DELL\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(fun

127.0.0.1 - - [21/Oct/2021 20:51:48] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [21/Oct/2021 20:51:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 20:51:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:15:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:15:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:16:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:16:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:16:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:16:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:16:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:16:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 21:16:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.